In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import shutil
import hashlib
from collections import OrderedDict
import pandas as pd
import time
import pickle
from utils import datasets

In [3]:
# # Try on small subset ..

# HEADER_ROW=['bag', 'time_sec', 'time_nsec', 'GPS', 'v_ego', 'key_event', 'frame_name']
# frames_dir = '/media/dean/datastore/datasets/kache_ai/frames_dev/'
# save_dir = os.path.join(frames_dir,'kache_set/')
# os.makedirs(save_dir, 0o755 , exist_ok = True )

# img_data = []
# lookup_table = {}
# img_dict = {}
# bag_path = os.path.join(frames_dir)
# img_dict['bag_path'] = frames_dir
# img_dict['frames'] = []
# img_dict['csv_path'] = os.path.join(bag_path, 'frames.csv')
# csv_logger = pd.read_csv(img_dict['csv_path'], names=HEADER_ROW, skiprows=1)

# # Get all images
# imgs = glob.glob(os.path.join(bag_path,'**/', '*.jpg'))

# for img in set(imgs):
#     d = {}
#     d['frame_path'] = img
#     hex_val = os.path.abspath(d['frame_path']).replace('/', '^^')+'---'+os.path.abspath(img_dict['csv_path']).replace('/', '^^')
#     hash_object = hashlib.sha1(str.encode(hex_val))
#     hex_dig = hash_object.hexdigest()
#     shutil.copyfile(d['frame_path'], os.path.join(save_dir, str(hex_dig)+'.jpg'))
#     d['dataset_path'] = os.path.join(save_dir, str(hex_dig)+'.jpg')
#     d['hash_key'] = hex_dig
#     d['hash_val'] = hex_val
#     lookup_table[d['hash_key']] = d['hash_val'] 
#     old_path = os.path.join(bag_path, 'frames', os.path.split(d['frame_path'])[-1])

#     for log in [x for x in csv_logger.as_matrix() if os.path.join(bag_path, 'frames', x[6].rstrip('/')) == old_path.rstrip('/')]:
#         d['bag_name'] = log[0]
#         d['time_sec'] = log[1]
#         d['time_readable'] = time.ctime(int(log[1]))

#         d['time_nsec'] = log[2]
#         d['GPS'] = log[3]
#         d['v-ego'] = log[4]

#         d['key_event'] = log[5]
#         d['frame'] = log[6]

#     if d.get('frame', None): img_dict['frames'].append(d)
# if img_dict['frames']: img_data.append((bag_path,img_dict))

# pickle_file = os.path.join(save_dir, 'kacheai_logs.pickle')
# pickle_dict = {'img_data':img_data,'lookup_table':lookup_table}
# print('Saving to Pickle File:', pickle_file)
# with open(pickle_file,"wb") as pickle_out:
#     pickle.dump(pickle_dict, pickle_out)



/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/ipykernel/__main__.py:33: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Saving to Pickle File: /media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/kacheai_logs.pickle


In [4]:
# print(img_data)

[('/media/dean/datastore/datasets/kache_ai/frames_dev/', {'bag_path': '/media/dean/datastore/datasets/kache_ai/frames_dev/', 'frames': [{'frame_path': '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0357.jpg', 'dataset_path': '/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/a76889152929741516cdfd751a726fe846907e58.jpg', 'hash_key': 'a76889152929741516cdfd751a726fe846907e58', 'hash_val': '^^media^^dean^^datastore^^datasets^^kache_ai^^frames_dev^^frames^^frame0357.jpg---^^media^^dean^^datastore^^datasets^^kache_ai^^frames_dev^^frames.csv', 'bag_name': 'bagfile_18-09-17_0724_156.bag', 'time_sec': 1537232681, 'time_readable': 'Mon Sep 17 18:04:41 2018', 'time_nsec': 929436922, 'GPS': '$GNGGA,010441.80,4039.73646,N,11220.78642,W,1,12,0.80,1284.8,M,-19.0,M,,*40\r', 'v-ego': 35.712501525878906, 'key_event': 'l', 'frame': 'frame0357.jpg'}, {'frame_path': '/media/dean/datastore/datasets/kache_ai/frames_dev/frames/frame0957.jpg', 'dataset_path': '/media/dean/datastor

In [5]:


# WORKING_DIR = '/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/'
# kache_set = datasets.DataFormatter(annotations_list = os.path.join(WORKING_DIR,'kacheai_logs.pickle'), image_list = WORKING_DIR, input_format = datasets.Format.kache,
#                                  output_path = os.path.join(WORKING_DIR,'scalabel_files/'),
#                                  trainer_prefix = 'COCO_train2014_0000', 
#                                  s3_bucket = 'kache-scalabel/kache_ai/frames/', check_s3 = False, use_static_categories = True)

DATA_CONFIG: /media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/scalabel_files/inference/cfg/bdd100k.data
MODEL_CONFIG: /media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/scalabel_files/inference/cfg/yolov3-bdd100k.cfg
MODEL_WEIGHTS: /media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/scalabel_files/inference/models/yolov3-bdd100k_51418.weights
Initializing Annotation Pipeline via Darknet ... \o/ \o/
DETECTION: (b'traffic sign', 0.962527334690094, (850.8175659179688, 547.1898803710938, 23.11958122253418, 27.990262985229492))
DETECTION: (b'car', 0.9041500091552734, (231.03758239746094, 583.8848876953125, 57.32524490356445, 34.01720428466797))
DETECTION: (b'traffic sign', 0.6798582673072815, (498.27569580078125, 558.0606079101562, 14.523720741271973, 22.086227416992188))
DETECTION: (b'traffic sign', 0.5985308289527893, (733.8927612304688, 553.1713256835938, 20.189857482910156, 21.395580291748047))
DETECTION: (b'car', 0.9747782349586487, (651.052001953125, 598

DETECTION: (b'traffic sign', 0.8129633665084839, (752.1627807617188, 559.1013793945312, 20.27505111694336, 19.058618545532227))
DETECTION: (b'car', 0.5658264756202698, (701.9371948242188, 583.0221557617188, 30.238353729248047, 23.336992263793945))
DETECTION: (b'car', 0.9375928044319153, (680.2110595703125, 595.4650268554688, 87.67716217041016, 66.24700927734375))
DETECTION: (b'car', 0.5387139320373535, (645.9777221679688, 578.355712890625, 18.645389556884766, 21.393335342407227))
DETECTION: (b'car', 0.9937255382537842, (662.5617065429688, 600.4056396484375, 108.52056884765625, 85.73237609863281))
DETECTION: (b'car', 0.8005945682525635, (596.4569702148438, 580.8319702148438, 32.19490432739258, 24.550548553466797))
DETECTION: (b'car', 0.6242980360984802, (754.172607421875, 571.2666015625, 29.995010375976562, 30.16057586669922))
DETECTION: (b'car', 0.929606556892395, (666.7007446289062, 592.8729858398438, 84.41021728515625, 66.79605102539062))
DETECTION: (b'car', 0.9939575791358948, (663.

DETECTION: (b'car', 0.9369008541107178, (365.0811767578125, 571.2020874023438, 101.02998352050781, 57.908939361572266))
DETECTION: (b'car', 0.9166285395622253, (675.9216918945312, 580.6112060546875, 63.455787658691406, 44.6968879699707))
DETECTION: (b'truck', 0.7168775796890259, (535.7941284179688, 554.1967163085938, 58.99138641357422, 43.41231918334961))
DETECTION: (b'car', 0.6536581516265869, (636.8834838867188, 559.1784057617188, 22.040939331054688, 20.81450653076172))
DETECTION: (b'car', 0.9876223802566528, (669.5311279296875, 591.692138671875, 82.71563720703125, 75.46935272216797))
DETECTION: (b'traffic sign', 0.636922299861908, (783.3667602539062, 537.9742431640625, 17.375812530517578, 18.808425903320312))
DETECTION: (b'car', 0.6079959273338318, (342.2186584472656, 578.1792602539062, 43.19179916381836, 25.005306243896484))
DETECTION: (b'traffic sign', 0.5552189946174622, (779.7400512695312, 544.4259643554688, 16.95975685119629, 15.953187942504883))
DETECTION: (b'car', 0.784029901

DETECTION: (b'car', 0.9897584915161133, (673.5911865234375, 601.5541381835938, 114.32476806640625, 93.18539428710938))
DETECTION: (b'car', 0.913520872592926, (855.7484130859375, 576.226318359375, 70.3309326171875, 49.24997329711914))
DETECTION: (b'car', 0.9082882404327393, (557.3594970703125, 576.3666381835938, 46.23616027832031, 38.9285888671875))
DETECTION: (b'car', 0.5294614434242249, (747.4597778320312, 570.0665893554688, 21.231842041015625, 17.638071060180664))
DETECTION: (b'car', 0.5184424519538879, (724.3427734375, 570.9402465820312, 19.328611373901367, 18.826553344726562))
DETECTION: (b'car', 0.9752655625343323, (660.5044555664062, 590.8721313476562, 71.04463195800781, 60.06461715698242))
DETECTION: (b'traffic sign', 0.9659361243247986, (780.3199462890625, 554.1953735351562, 39.54949951171875, 16.421463012695312))
DETECTION: (b'car', 0.9710273742675781, (673.5848999023438, 591.7776489257812, 67.51681518554688, 49.461753845214844))
DETECTION: (b'car', 0.9203510284423828, (708.38

DETECTION: (b'car', 0.9967224597930908, (661.1170654296875, 600.7432861328125, 91.94680786132812, 82.51409912109375))
DETECTION: (b'traffic sign', 0.9259227514266968, (909.8438110351562, 493.4818420410156, 69.60650634765625, 65.94017791748047))
DETECTION: (b'traffic sign', 0.7850687503814697, (1154.039794921875, 553.0386962890625, 93.17788696289062, 111.24429321289062))
DETECTION: (b'car', 0.7368117570877075, (608.1397094726562, 575.7666625976562, 33.76531219482422, 28.77791976928711))
DETECTION: (b'traffic sign', 0.9755292534828186, (952.4771118164062, 527.719482421875, 23.677440643310547, 27.63837432861328))
DETECTION: (b'car', 0.9639559984207153, (699.845458984375, 591.1693115234375, 90.30541229248047, 65.68946838378906))
DETECTION: (b'traffic sign', 0.8662585616111755, (1066.273193359375, 527.6777954101562, 18.57384490966797, 18.442228317260742))
DETECTION: (b'car', 0.9613620042800903, (668.183837890625, 591.3677368164062, 77.35433197021484, 64.23180389404297))
DETECTION: (b'car', 

...

DETECTION: (b'car', 0.9290233850479126, (676.0021362304688, 586.9637451171875, 85.55015563964844, 65.14575958251953))
DETECTION: (b'car', 0.9932355880737305, (666.8563842773438, 596.801513671875, 95.63616943359375, 81.11109924316406))
DETECTION: (b'traffic sign', 0.9616295099258423, (966.0730590820312, 476.31463623046875, 83.2680892944336, 67.50047302246094))
DETECTION: (b'car', 0.7469876408576965, (601.67333984375, 574.1124267578125, 26.50498390197754, 23.826126098632812))
DETECTION: (b'car', 0.9901180863380432, (299.6536560058594, 582.4630737304688, 79.11284637451172, 37.91131591796875))
DETECTION: (b'truck', 0.9220615029335022, (646.9424438476562, 550.7127685546875, 72.11874389648438, 95.61497497558594))
DETECTION: (b'truck', 0.806462824344635, (849.2186889648438, 559.4730834960938, 100.53275299072266, 110.62332153320312))
DETECTION: (b'truck', 0.7961348295211792, (994.0823364257812, 531.3058471679688, 227.14593505859375, 217.7889862060547))
DETECTION: (b'car', 0.9672703146934509, (

DETECTION: (b'car', 0.9623728394508362, (132.0572509765625, 620.1278686523438, 267.43804931640625, 385.20513916015625))
DETECTION: (b'car', 0.912470281124115, (708.4791259765625, 576.5811157226562, 85.30302429199219, 78.56240844726562))
DETECTION: (b'traffic sign', 0.8098816871643066, (879.0860595703125, 521.2566528320312, 20.430347442626953, 23.976192474365234))
DETECTION: (b'truck', 0.6935417056083679, (496.51953125, 529.6333618164062, 151.716064453125, 74.91326904296875))
DETECTION: (b'car', 0.6484639644622803, (785.5007934570312, 517.80517578125, 24.02389144897461, 18.9360408782959))
DETECTION: (b'traffic sign', 0.737216591835022, (889.19189453125, 546.5130004882812, 32.568397521972656, 22.72968101501465))
DETECTION: (b'car', 0.6414358019828796, (722.7251586914062, 579.3028564453125, 54.41783905029297, 43.37830352783203))
DETECTION: (b'car', 0.9296070337295532, (666.5711669921875, 591.6281127929688, 81.11701202392578, 61.794837951660156))
DETECTION: (b'car', 0.996337890625, (660.53

DETECTION: (b'car', 0.9963144063949585, (382.7435607910156, 586.0963745117188, 228.19973754882812, 166.63638305664062))
DETECTION: (b'car', 0.8851091265678406, (702.1265258789062, 578.6072998046875, 83.79935455322266, 64.79322052001953))
DETECTION: (b'traffic sign', 0.8671596646308899, (956.8228759765625, 519.6181640625, 59.310943603515625, 50.49500274658203))
DETECTION: (b'traffic sign', 0.7894158363342285, (888.4155883789062, 507.96392822265625, 35.39606475830078, 43.76301193237305))
DETECTION: (b'car', 0.926450252532959, (676.1259765625, 583.6558227539062, 60.1903076171875, 40.209232330322266))
DETECTION: (b'car', 0.9085787534713745, (1022.5333251953125, 579.033203125, 74.99480438232422, 62.11681365966797))
DETECTION: (b'car', 0.814845860004425, (896.3858642578125, 573.8685302734375, 73.54961395263672, 46.257568359375))
DETECTION: (b'truck', 0.7440089583396912, (1115.28759765625, 576.97998046875, 144.61651611328125, 82.94277954101562))
DETECTION: (b'car', 0.6884896159172058, (963.01

DETECTION: (b'car', 0.9787389636039734, (636.9379272460938, 592.1873168945312, 82.75115203857422, 59.610862731933594))
DETECTION: (b'truck', 0.6906496286392212, (1035.4227294921875, 469.98291015625, 572.1795043945312, 547.0108032226562))
DETECTION: (b'car', 0.9724457263946533, (93.14118194580078, 607.1387329101562, 122.147216796875, 56.430110931396484))
DETECTION: (b'car', 0.9564324021339417, (691.0460205078125, 584.46533203125, 70.99752044677734, 56.606101989746094))
DETECTION: (b'car', 0.846041738986969, (648.9279174804688, 571.5192260742188, 27.199111938476562, 23.50431251525879))
DETECTION: (b'traffic sign', 0.7821252942085266, (1096.7467041015625, 459.921630859375, 49.36708068847656, 66.93734741210938))
DETECTION: (b'traffic sign', 0.6829062104225159, (728.8280639648438, 452.0942077636719, 31.753074645996094, 19.557254791259766))
DETECTION: (b'car', 0.5304607152938843, (449.3402404785156, 577.6651611328125, 61.26464080810547, 29.282493591308594))
DETECTION: (b'traffic sign', 0.937

DETECTION: (b'car', 0.6981351375579834, (675.4226684570312, 575.9561767578125, 27.3134822845459, 26.267845153808594))
DETECTION: (b'car', 0.500138521194458, (630.3729248046875, 573.8945922851562, 26.99995994567871, 30.47947120666504))
DETECTION: (b'car', 0.9653763771057129, (680.9229736328125, 579.8798828125, 55.041358947753906, 41.25768280029297))
DETECTION: (b'car', 0.962550163269043, (1215.198486328125, 584.2977905273438, 125.79454040527344, 105.57120513916016))
DETECTION: (b'car', 0.8738684058189392, (1101.9566650390625, 592.705078125, 108.54042053222656, 60.42188262939453))
DETECTION: (b'car', 0.5338164567947388, (598.2074584960938, 576.0368041992188, 20.625469207763672, 21.5286865234375))
DETECTION: (b'truck', 0.5059473514556885, (574.6190795898438, 567.4549560546875, 46.290584564208984, 41.983795166015625))
Saving to Pickle File: kache_ai_frames_dev_kache_set_kacheai_logs.pickle.pickle
Length of COCO Images 1909
Length of COCO Annotations: 5635
loading annotations into memory...

In [6]:
# kache_set.export(format = datasets.Format.scalabel, paginate = True)

In [7]:
HEADER_ROW=['bag', 'time_sec', 'time_nsec', 'GPS', 'v_ego', 'key_event', 'frame_name']
frames_dir = '/media/dean/datastore/datasets/kache_ai/frames/'
bag_dirs = os.listdir(frames_dir)
save_dir = os.path.join(frames_dir,'kache_set/')
os.makedirs(save_dir, 0o755 , exist_ok = True )

img_data = []
lookup_table = {}

for bag in bag_dirs:
    if frames_dir != save_dir or '.csv' not in frames_dir:
        img_dict = {}
        bag_path = os.path.join(frames_dir, bag)
        img_dict['bag_path'] = frames_dir
        img_dict['frames'] = []
        img_dict['csv_path'] = os.path.join(bag_path, 'frames.csv')
        if os.path.exists(img_dict['csv_path']):
            csv_logger = pd.read_csv(img_dict['csv_path'], names=HEADER_ROW, skiprows=1)

            # Get all images
            imgs = glob.glob(os.path.join(bag_path,'**/', '*.jpg'))

            for img in set(imgs):
                d = {}
                d['frame_path'] = img
                hex_val = os.path.abspath(d['frame_path']).replace('/', '^^')+'---'+os.path.abspath(img_dict['csv_path']).replace('/', '^^')
                hash_object = hashlib.sha1(str.encode(hex_val))
                hex_dig = hash_object.hexdigest()
                shutil.copyfile(d['frame_path'], os.path.join(save_dir, str(hex_dig)+'.jpg'))
                d['dataset_path'] = os.path.join(save_dir, str(hex_dig)+'.jpg')
                d['hash_key'] = hex_dig
                d['hash_val'] = hex_val
                lookup_table[d['hash_key']] = d['hash_val'] 
                old_path = os.path.join(bag_path, 'frames', os.path.split(d['frame_path'])[-1])

                for log in [x for x in csv_logger.as_matrix() if os.path.join(bag_path, 'frames', x[6].rstrip('/')) == old_path.rstrip('/')]:
                    d['bag_name'] = log[0]
                    d['time_sec'] = log[1]
                    d['time_readable'] = time.ctime(int(log[1]))

                    d['time_nsec'] = log[2]
                    d['GPS'] = log[3]
                    d['v-ego'] = log[4]

                    d['key_event'] = log[5]
                    d['frame'] = log[6]

                if d.get('frame', None): img_dict['frames'].append(d)
            if img_dict['frames']: img_data.append((bag_path,img_dict))

pickle_file = os.path.join(save_dir, 'kacheai_logs.pickle')
pickle_dict = {'img_data':img_data,'lookup_table':lookup_table}
print('Saving to Pickle File:', pickle_file)
with open(pickle_file,"wb") as pickle_out:
    pickle.dump(pickle_dict, pickle_out)



In [8]:
WORKING_DIR = '/media/dean/datastore/datasets/kache_ai/frames/kache_set/'
kache_set = datasets.DataFormatter(annotations_list = '/media/dean/datastore/datasets/kache_ai/frames/kache_set/kacheai_logs.pickle', image_list = '/media/dean/datastore/datasets/kache_ai/frames/kache_set/', input_format = datasets.Format.kache,
                                 output_path = os.path.join(WORKING_DIR,'scalabel_files'),
                                 trainer_prefix = 'COCO_train2014_0000', 
                                 s3_bucket = 'kache-scalabel/kache_ai/frames/', check_s3 = False, use_static_categories = True)

In [9]:
kache_set.export(format = datasets.Format.scalabel, paginate = True, use_static_categories = True)